In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler


# Carregue seus dados em um DataFrame
data = pd.read_excel('Dataset/Dados da Torre de Santarém com NPP.xlsx')


# Divida os dados em recursos (X) e rótulo (y)
X = data[['Tair_f', 'VPD_f', 'rH', 'Year', 'H_f','FP_Temp_uStar','Tsoil_f', 'LE_f','Rg_f']]
y = data['NPP']


# Padronize os recursos
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


# Treine o modelo GBM
gbm = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=4, random_state=42)
gbm.fit(X_scaled, y)


# Calcule a importância das características
feature_importance = gbm.feature_importances_


mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)


print(f'MSE (Mean Squared Error): {mse}')
print(f'MAE (Mean Absolute Error): {mae}')
print(f'R-squared (R2 Score): {r2}')




# Crie um gráfico de barras para mostrar a importância das características
plt.bar(X.columns, feature_importance)
plt.xlabel('Variáveis')
plt.ylabel('Importância')
plt.title('Importância das Variáveis em Relação ao NPP')
plt.xticks(rotation=45)
plt.show()


ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler


# Carregue seus dados em um DataFrame
data = pd.read_excel('Dataset/Dados da Torre de Santarém com NPP.xlsx')


# Divida os dados em recursos (X) e rótulo (y)
X = data[['Tair_f', 'VPD_f', 'rH', 'Year', 'H_f','FP_Temp_uStar','Tsoil_f', 'LE_f','Rg_f']]
y = data['NPP']


# Padronize os recursos
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


# Treine o modelo GBM
gbm = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=4, random_state=42)
gbm.fit(X_scaled, y)


# Calcule a importância das características
feature_importance = gbm.feature_importances_


mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)


print(f'MSE (Mean Squared Error): {mse}')
print(f'MAE (Mean Absolute Error): {mae}')
print(f'R-squared (R2 Score): {r2}')




# Crie um gráfico de barras para mostrar a importância das características
plt.bar(X.columns, feature_importance)
plt.xlabel('Variáveis')
plt.ylabel('Importância')
plt.title('Importância das Variáveis em Relação ao NPP')
plt.xticks(rotation=45)
plt.show()
